In [12]:
%pip install pydub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import yaml
import os
from pydub import AudioSegment
#from demucs.train import get_datasets 
sources = ['vocals', 'drums', 'bass', 'guitar', 'other']
path = r'C:\Users\justm\Downloads\babyslakh_16k.tar\babyslakh_16k'
folders = os.listdir(path)
output_path = os.path.join(path, 'test')
os.makedirs(output_path, exist_ok=True)
temp_dict = {}


def combine_audios(source, paths, output_location):
    if not paths:
        return None
    if os.path.exists(paths[0]):
        combined = AudioSegment.from_wav(paths[0])
    else:
        combined = AudioSegment.silent(duration=0)
    for p in paths[1:]:
        try:
            audio = AudioSegment.from_wav(p)
            
        except FileNotFoundError:
            print(f"File not found: {p}")
            continue
        combined = combined.overlay(audio)
    out_path = os.path.join(output_path,output_location)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    out_path = os.path.join(out_path, f"{source}.wav")
    result = combined.export(out_path, format='wav')
    return result


for folder in folders:
    folder_path = os.path.join(path, folder)
    if os.path.isdir(folder_path):
        files = os.listdir(folder_path)
        if 'test' in folders:
            folders.remove('test')
            continue
        
        for file in files:
            
            if file == 'metadata.yaml':
                meta_path = os.path.join(folder_path, file)
                with open(meta_path, 'r', encoding='utf-8') as f:
                    meta = yaml.safe_load(f)
                temp_dict = {}
                for st in meta['stems'].keys():
                    name = st
                    source = meta['stems'][st]['inst_class'].lower()
                    if source not in sources:
                        source = 'other'
                    wav_path = os.path.join(folder_path,'stems', st + '.wav')
                    if temp_dict.get(source) is None:
                        temp_dict[source] = {'paths': [wav_path]}
                    else:
                        temp_dict[source]['paths'].append(wav_path)
                for s in sources:
                    if temp_dict.get(s) is None:
                        continue
                    else:
                        if len(temp_dict[s]['paths']) > 0:
                            # Combine the audio files for this source
                            print(f"Combining {s} audio files...")
                            # Combine the audio files for this source
                            new_out = combine_audios(s, temp_dict[s]['paths'], folder)
                            # Clear the paths list after combining
                            temp_dict[s]['paths'] = None
                    new_out = combine_audios(s, temp_dict[s]['paths'], folder)

                break

    
